# Proyecto final 2: Detección de SPAM con Machine Learning

### Enunciado y contexto del ejercicio

Se propone la construcción de un sistema de aprendizaje automático capaz de predecir si un correo determinado se corresponde con un correo de SPAM o no, para ello, se utilizará el siguiente conjunto de datos:


### Conjunto de datos

##### [2007 TREC Public Spam Corpus](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo07)
The corpus trec07p contains 75,419 messages:

    25220 ham
    50199 spam

These messages constitute all the messages delivered to a particular
server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

### 1. Lectura de los correos electrónicos

Antes de comenzar a realizar ninguna acción, debemos implementar una función que lea de disco los correos electrónicos que forman parte del conjunto de datos de manera que podamos visualizar el formato que tienen.

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Implementa una función en Python que lea el conjunto de datos de correos electrónicos y las etiquetas asociados a ellos.
</div>

**Pista**: Revisa los archivos que se encuentran en el directorio del conjunto de datos, el archivo `full/index` contiene la etiqueta de los correos y la ruta al mismo.

In [2]:
# Si usamos esta sentencia "with", no tenemos que preocuparnos de cerrar el archivo manualmente llamando al método "close", es decir,
# la sentencia "with" se encarga de cerrar el archivo automáticamente.
with open("full/index", "r") as f:
    print(dir(f))
    #print(f.readlines())

['_CHUNK_SIZE', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_finalizing', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'reconfigure', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'write_through', 'writelines']


In [1]:
def leer_correos(indice, num):
    correos = []
    etiquetas = []
    with open(indice, "r") as f_indice:
        for i in range(num):
            # "[:-1]" -> Para eliminar el caracter de salto de línea '\n'
            linea = f_indice.readline()[:-1]
            etiqueta, ruta_correo = linea.split(" ../")
            etiquetas.append(etiqueta)
            # Usamos el valor "ignore" en el argumento "errors" porque los correos pueden contener caracteres con una codificación extraña y no
            # queremos que falle la lectura del correo debido a esos caracteres.
            with open(ruta_correo, "r", errors = "ignore") as f_correo:
                correos.append(f_correo.read())
    return correos, etiquetas

In [19]:
correos, etiquetas = leer_correos("full/index", 10)

In [10]:
print(correos[0])

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

In [12]:
print(etiquetas[0])

spam


### 2. Procesamiento de texto HTML en los emails

En este caso práctico relacionado con la detección de correos electrónicos de SPAM, el conjunto de datos que disponemos esta formado por correos electrónicos, con sus correspondientes cabeceras y campos adicionales. Por lo tanto, requieren un preprocesamiento previo a que sean ingeridos por el algoritmo de Machine Learning.

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Implementa una clase en Python 3 que permita procesar texto que contiene código HTML y elimine los tags HTML.
</div>

**Pista**: Revisa la clase `HTMLParser` para implementar esta sección: https://docs.python.org/es/3.8/library/html.parser.html

In [5]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):

    def __init__(self):
        super().__init__()
        self._data = []

    def handle_data(self, data):
        self._data.append(data)

    def get_data(self):
        return "".join(self._data)

In [35]:
analizador_html = MyHTMLParser()

In [36]:
print(correos[0])

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

In [37]:
analizador_html.feed(correos[0])

In [38]:
print(analizador_html.get_data())

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: 
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: 
From: "Tomas Jacobs" 
Reply-To: "Tomas Jacobs" 
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit








Do you feel the pressure to perform and not rising to the occasion??





Try Viagra.....
your anxiety will be a thing of the past and you will
be back to your old self.



----8896484051606557286--




In [6]:
def eliminar_etiquetas_html(html):
    analizador_html = MyHTMLParser()
    analizador_html.feed(html)
    return analizador_html.get_data()

In [43]:
print(eliminar_etiquetas_html(correos[0]))

From bounce-debian-mirrors=ktwarwic=speedy.uwaterloo.ca@lists.debian.org  Sun Apr  8 13:09:29 2007
Return-Path: 
Received: from murphy.debian.org (murphy.debian.org [70.103.162.31])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with ESMTP id l38H9S0I003031
	for ; Sun, 8 Apr 2007 13:09:28 -0400
Received: from localhost (localhost [127.0.0.1])
	by murphy.debian.org (Postfix) with QMQP
	id 90C152E68E; Sun,  8 Apr 2007 12:09:05 -0500 (CDT)
Old-Return-Path: 
X-Spam-Checker-Version: SpamAssassin 3.1.4 (2006-07-26) on murphy.debian.org
X-Spam-Level: 
X-Spam-Status: No, score=-1.1 required=4.0 tests=BAYES_05 autolearn=no 
	version=3.1.4
X-Original-To: debian-mirrors@lists.debian.org
Received: from xenon.savoirfairelinux.net (savoirfairelinux.net [199.243.85.90])
	by murphy.debian.org (Postfix) with ESMTP id 827432E3E5
	for ; Sun,  8 Apr 2007 11:52:35 -0500 (CDT)
Received: from [192.168.0.101] (bas6-montreal28-1177925679.dsl.bell.ca [70.53.184.47])
	by xenon.savoirfairelinux.net (Postfix) with ESMTP 

In [44]:
print(eliminar_etiquetas_html(correos[1]))

From bounce-debian-mirrors=ktwarwic=speedy.uwaterloo.ca@lists.debian.org  Sun Apr  8 13:09:29 2007
Return-Path: 
Received: from murphy.debian.org (murphy.debian.org [70.103.162.31])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with ESMTP id l38H9S0I003031
	for ; Sun, 8 Apr 2007 13:09:28 -0400
Received: from localhost (localhost [127.0.0.1])
	by murphy.debian.org (Postfix) with QMQP
	id 90C152E68E; Sun,  8 Apr 2007 12:09:05 -0500 (CDT)
Old-Return-Path: 
X-Spam-Checker-Version: SpamAssassin 3.1.4 (2006-07-26) on murphy.debian.org
X-Spam-Level: 
X-Spam-Status: No, score=-1.1 required=4.0 tests=BAYES_05 autolearn=no 
	version=3.1.4
X-Original-To: debian-mirrors@lists.debian.org
Received: from xenon.savoirfairelinux.net (savoirfairelinux.net [199.243.85.90])
	by murphy.debian.org (Postfix) with ESMTP id 827432E3E5
	for ; Sun,  8 Apr 2007 11:52:35 -0500 (CDT)
Received: from [192.168.0.101] (bas6-montreal28-1177925679.dsl.bell.ca [70.53.184.47])
	by xenon.savoirfairelinux.net (Postfix) with ESMTP 

### 3. Procesamiento de lenguaje natural

Además de eliminar los posibles tags HTML que se encuentren en el correo electrónico, deben realizarse otras acciones de preprocesamiento para evitar que los mensajes contengan ruido innecesario. Entre ellas se encuentra la eliminación de los signos de puntuación, eliminación de posibles campos del correo electrónico que no son relevantes o eliminación de los afijos de una palabra manteniendo únicamente la raiz de la misma (Stemming).

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Explora e implementa diferentes funciones en Python 3 que permitan realizar los procesamientos que se indican en el texto anterior. Ten en cuenta que el texto de los correos electrónicos esta en Inglés.
</div>

**Pista 1**: Ten en cuenta que los correos electrónicos se encuentran en bruto y, por lo tanto, contienen valores que no nos van a resultar de interés, por ejemplo, las cabeceras o el pie del correo electrónico. Utiliza el paquete externo `email` para procesar los correos y eliminar todo menos el cuerpo del mismo.

In [5]:
import email

In [9]:
correo_mensaje = email.message_from_string(correos[0])
correo_payloads = correo_mensaje.get_payload()

In [10]:
correo_payloads

In [14]:
print(eliminar_etiquetas_html(correo_payloads[0].get_payload()))








Do you feel the pressure to perform and not rising to the occasion??





Try Viagra.....
your anxiety will be a thing of the past and you will
be back to your old self.





In [25]:
def obtener_payloads(correo):
    def parsear_payload(payload):
        if type(payload) is str:
            return [payload]
        else:
            payloads = []
            for p in payload:
                payloads += parsear_payload(p.get_payload())
            return payloads
    correo_mensaje = email.message_from_string(correo)
    return parsear_payload(correo_mensaje.get_payload())

In [28]:
obtener_payloads(correos[2])

['Mega  authenticV I A G R A   $ DISCOUNT priceC I A L I S  $DISCOUNT priceDo not miss IT, CLICK here.\nhttp://www.moujsjkhchum.com\n\n',
 '<HTML><HEAD><TITLE>authentic viagra</TITLE></HEAD>\n<BODY>\nMega  authentic<br>V I A G R A   $ DISCOUNT price<br>C I A L I S  $DISCOUNT price<br><a href="http://www.moujsjkhchum.com">Do not miss IT, CLICK here.</a>\n</BODY></HTML>\n']

**Pista 2**: Una vez que hayas eliminado todos los componentes del correo menos su cuerpo, explora la librería `nltk` para eliminar signos de puntuación y afijos. Revisa la clase `PorterStemmer()` de nltk así como los métodos y atributos `nltk.corpus.stopwords.words('english')`, `string.punctuation` y `nltk.tokenize.word_tokenize`

In [1]:
import nltk

In [2]:
stemmer = nltk.PorterStemmer()

In [3]:
# Obtiene la raíz del verbo indicado(infinitivo)
stemmer.stem("doing")

'do'

In [4]:
stemmer.stem("making")

'make'

In [5]:
stemmer.stem("have")

'have'

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manuel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
# Obtiene palabras en el idioma indicado que carecen de significado por si solas
nltk.corpus.stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [11]:
import string

# Devuelve un string con caracteres o símbolos de puntuación(son caracteres que carecen de significado)
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
from nltk.tokenize import word_tokenize

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manuel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [17]:
# Separa las palabras de los símbolos de puntuación
word_tokenize("Hola como estas; me llamo juan...")

['Hola', 'como', 'estas', ';', 'me', 'llamo', 'juan', '...']

### 4. Procesamiento de texto HTML en los emails (II)

Pon todo lo anterior en común y construye una clase `EmailParser` que procese los emails aplicando todas las transformaciones que hemos implementado en las secciones anteriores. 

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Implementa una clase en Python 3 que contenga toda la funcionalidad que hemos visto en el apartado anterior. Implementa una función que permita leer correos electrónicos y aplicarles las transformaciones de manera simultánea.
</div>

In [2]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):

    def __init__(self):
        super().__init__()
        self._data = []

    def handle_data(self, data):
        self._data.append(data)

    def get_data(self):
        return "".join(self._data)

In [3]:
import email
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk import PorterStemmer, download
from nltk.corpus import stopwords

download('punkt')
download('stopwords')

class AnalizadorCorreos:

    def analizar(self, correo):
        # Obtenemos los cuerpos, o payloads, del correo
        str_payloads = "".join(self._obtener_payloads(correo))
        #print("_obtener_payloads", str_payloads)
        # Eliminamos las etiquetas HTML de los cuerpos, o payloads, del correo
        str_payloads = self._eliminar_etiquetas_html(str_payloads)
        #print("_eliminar_etiquetas_html", str_payloads)
        # Eliminamos las urls de los cuerpos, o payloads, del correo
        str_payloads = self._eliminar_urls(str_payloads)
        # Separamos los cuerpos, o payloads, del correo en tokens
        palabras_payloads = word_tokenize(str_payloads)
        # Eliminamos los tokens o palabras que sean "stopwords" y puntuaciones y, además, aplicamos "stemming"
        palabras_payloads = self._limipiar_palabras(palabras_payloads)
        return " ".join(palabras_payloads)

    def _parsear_payload(self, payload):
        if type(payload) is str:
            return [payload]
        else:
            payloads = []
            for p in payload:
                payloads += self._parsear_payload(p.get_payload())
            return payloads
    
    def _obtener_payloads(self, correo):
        correo_mensaje = email.message_from_string(correo)
        return self._parsear_payload(correo_mensaje.get_payload())

    def _eliminar_etiquetas_html(self, payloads):
        analizador_html = MyHTMLParser()
        analizador_html.feed(payloads)
        return analizador_html.get_data()

    def _eliminar_urls(self, payloads):
        return re.sub(r"http\S+", "", payloads)

    def _limipiar_palabras(self, palabras):
        ps = PorterStemmer()
        puntuaciones = list(punctuation) + ['\n', '\t']
        palababras_limpias = []
        for palabra in palabras:
            if palabra not in stopwords.words("english") and palabra not in puntuaciones:
                palababras_limpias.append(ps.stem(palabra))
        return palababras_limpias;

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
correos[0]

NameError: name 'correos' is not defined

In [49]:
analizador_correos = AnalizadorCorreos()

In [50]:
analizador_correos.analizar(correos[0])

'do feel pressur perform rise occas tri viagra ..... anxieti thing past back old self'

In [51]:
analizador_correos.analizar(correos[1])

"hi 've updat gulu i check mirror it seem littl typo /debian/readm file exampl ftp //ftp.fr.debian.org/debian/readm '' test lenni access releas dists/test the current test develop snapshot name etch packag test unstabl pass autom test propog releas '' etch replac lenni like readme.html -- yan morin consult en logiciel libr yan.morin savoirfairelinux.com 514-994-1556 -- to unsubscrib email debian-mirrors-request lists.debian.org subject `` unsubscrib '' troubl contact listmast lists.debian.org"

In [4]:
def crear_dataset(indice, num):
    analizador_correos = AnalizadorCorreos()
    correos, etiquetas = leer_correos(indice, num)
    correos_analizados = []
    for i, correo in zip(range(len(correos)), correos):
        print("\rAnalizando correo: {0}".format(i + 1), end = "")
        correos_analizados.append(analizador_correos.analizar(correo))
    return correos_analizados, etiquetas

In [8]:
correos_procesados, etiquetas = crear_dataset("full/index", 30)

Analizando correo: 30

In [9]:
correos_procesados[0]

'do feel pressur perform rise occas tri viagra ..... anxieti thing past back old self'

In [10]:
etiquetas[0]

'spam'

In [11]:
correos_procesados[20]

"gruess v.harishankar schrieb 08.04.07 06:49 i problem hp compaq nx6110 laptop sinc upgrad kernel 2.6.18-4-686 current run version 2.6.18.dfsg.1-12 debian etch i 've unabl get acpi work machin as result i n't get batteri ac statu machin n't power properli i shut if alreadi seen regard hari gruß gerhard -- kernel panic could determin whether bit one zero sqrt 1/pi ... j.k. d-u-g -- to unsubscrib email debian-laptop-request lists.debian.org subject `` unsubscrib '' troubl contact listmast lists.debian.org"

### 5. Codificando el conjunto de datos

Con las funciones presentadas anteriormente se permite la lectura de los correos electrónicos de manera programática y el procesamiento de los mismos para eliminar aquellos componentes que no resultan de utilidad para la detección de correos de SPAM. Sin embargo, cada uno de los correos sigue estando representado por un `string`.

La mayoría de los algoritmos de Machine Learning no son capaces de ingerir texto como parte del conjunto de datos. Por lo tanto, deben aplicarse una serie de funciones adicionales que transformen el texto de los correos electrónicos parseados en una representación numérica.

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Aplica alguna técnica de codificación/vectorización sobre los correos electrónicos parseados para transformar el texto en una representación numérica.
</div>

**Pista**: Revisa la clase `CountVectorizer` de Sklearn para realizar la codificación.

In [12]:
correos_procesados[0]

'do feel pressur perform rise occas tri viagra ..... anxieti thing past back old self'

In [15]:
correos_procesados[2]

'mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click authent viagra mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click'

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
count_vectorizer = CountVectorizer()

In [24]:
count_vectorizer.fit([correos_procesados[0], correos_procesados[2], correos_procesados[10]])

CountVectorizer()

In [25]:
count_vectorizer.get_feature_names_out()

array(['15', '38', '44', '62', '85', 'action', 'anxieti', 'as', 'authent',
       'authenticv', 'avail', 'back', 'be', 'best', 'br', 'cia', 'ciali',
       'click', 'custom', 'day', 'discount', 'do', 'doctor', 'dose',
       'drug', 'drugstor', 'feel', 'for', 'free', 'full', 'good', 'it',
       'levitra', 'low', 'medic', 'mega', 'minut', 'miss', 'most', 'much',
       'need', 'new', 'no', 'occas', 'offer', 'old', 'onlin', 'past',
       'per', 'perform', 'prescript', 'pressur', 'pricec', 'pricedo',
       'prope', 'propecia', 'readi', 'regard', 'rise', 'satisfact',
       'satisfactionclick', 'save', 'self', 'ship', 'special', 'store',
       'super', 'thing', 'to', 'today', 'tri', 'upto', 'viagra', 'visit',
       'worlwid', 'you'], dtype=object)

In [29]:
vector = count_vectorizer.transform([correos_procesados[2]])
vector.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0]], dtype=int64)

In [30]:
correos_procesados[0]

'do feel pressur perform rise occas tri viagra ..... anxieti thing past back old self'

### 6. Entrenamiento del algoritmo 

¡Enhorabuena! Ya tienes la parte más complicada del ejercicio realizada. En este apartado vamos a entrenar un algoritmo de Machine Learning que aprenderá de los vectores anteriores a clasificar los correos en spam y legítimos.

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Utiliza el algoritmo de Machine Learning LogisticRegression para clasificar entre correos electrónicos de spam y legítimos. Puedes encontrar la implementación de este algoritmo en Sklearn.
</div>

**Pista 1**: Comienza leyendo un número de correos pequeño para que no tarde demasiado tiempo, por ejemplo 100 correos. Aplica todas las transformaciones que hemos implementado anteriormente sobre ellos.

In [31]:
correos_procesados, etiquetas = crear_dataset("full/index", 100)

Analizando correo: 100

**Pista 2:** Aplica la vectorización al conjunto de datos para representar los correos de manera numérica.

In [32]:
count_vectorizer = CountVectorizer()

In [33]:
count_vectorizer.fit(correos_procesados)

CountVectorizer()

In [34]:
correos_vectorizados = count_vectorizer.transform(correos_procesados)

In [35]:
correos_vectorizados.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [36]:
import pandas as pd

pd.DataFrame(correos_vectorizados.toarray(), columns = count_vectorizer.get_feature_names_out())

,00,000,0000,000000,000099,0001pt,000m,0085,009,00w6yil7xydn0ph8kc0jqlu11p00mybdlyojxkg5qmk,...,õôõôèõéï,ö¹,öð,öôööµæ,öø³ðåµ,öþ,öˆ,úàí,þîñòµ¼,šè
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Pista 3**: Entrena el algoritmo `LogisticRegression` de Sklearn. Este algoritmo se basa en aprendizaje supervisado y funciona exactamente igual que el algoritmo `Perceptron` que hemos presentado en secciones anteriores.

In [39]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
logistic_regression.fit(correos_vectorizados, etiquetas)

LogisticRegression()

### 7. Predicción

¡Muy bien, ya casi has terminado! Ya tenemos nuestro algoritmo entrenado y listo para realizar predicciones, lo único que nos queda es probar que tal se comporta para correos que no ha visto nunca.

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Utiliza el algoritmo de Machine Learning LogisticRegression para predecir si un nuevo correo que no ha visto nunca es spam o legitimo.
</div>

**Pista**: Revisa el método `predict()` que tiene la clase `LogisticRegression`. Ten en cuenta que cuando recibas un nuevo correo para el que quieres realizar una predicción debes aplicar sobre él todas las transformaciones que hemos realizado anteriormente.  **Al aplicar la vectorización utiliza únicamente el método `transform()` del objeto `CountVectorizer`**.

In [40]:
correos_procesados, etiquetas = crear_dataset("full/index", 150)

Analizando correo: 150

In [42]:
correos_prueba = correos_procesados[100:]
etiquetas_prueba = etiquetas[100:]

In [43]:
len(correos_prueba)

50

In [44]:
correos_vectorizados = count_vectorizer.transform(correos_prueba)

In [45]:
correos_vectorizados.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [53]:
etiquetas_prediccion = logistic_regression.predict(correos_vectorizados)

In [54]:
print("\nPredicción:\n", etiquetas_prediccion)
print("\nEtiquetas reales:\n", etiquetas_prueba)


Predicción:
 ['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'ham' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam']

Etiquetas reales:
 ['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam']


**Pista**: Utiliza una métrica para evaluar los resultados. En este caso, utiliza la métrica `accuracy_score`. Puedes encontrar esta métrica implementada en Sklearn.

In [61]:
from sklearn.metrics import accuracy_score

print("Accuracy: {:.3f}".format(accuracy_score(etiquetas_prueba, etiquetas_prediccion)))

Accuracy: 0.940


### 8. Aumentando el conjunto de datos

Nuestro filtro de SPAM ya funciona realmente bien, nos ha dado una precision del 94% y practicamente no se ha equivocado en la clasificaciones. Sin embargo, este tipo de algoritmos suelen funcionar mejor con conjuntos de datos mas grandes. Prueba a entrenar el algoritmo con un conjunto de datos de entrenamiento mas grande.

<div style="background-color:#D9EEFF;color:black;padding:2%;">
Repite el ejercicio anterior utilizando un conjunto de datos de entrenamiento más grande.
</div>

In [7]:
correos_procesados, etiquetas = crear_dataset("full/index", 5000)

Analizando correo: 5000

In [8]:
correos_entrenamiento = correos_procesados[:4000]
etiquetas_entrenamiento = etiquetas[:4000]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

In [10]:
count_vectorizer.fit(correos_entrenamiento)

CountVectorizer()

In [11]:
correos_vectorizados = count_vectorizer.transform(correos_entrenamiento)

In [12]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()

In [13]:
logistic_regression.fit(correos_vectorizados, etiquetas_entrenamiento)

C:\Users\manuel\anaconda3\envs\curso-python\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [14]:
correos_prueba = correos_procesados[4000:]
etiquetas_prueba = etiquetas[4000:]

In [15]:
correos_vectorizados = count_vectorizer.transform(correos_prueba)

In [16]:
etiquetas_prediccion = logistic_regression.predict(correos_vectorizados)

In [17]:
print("\nPredicción:\n", etiquetas_prediccion)
print("\nEtiquetas reales:\n", etiquetas_prueba)


Predicción:
 ['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'ham' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam' 'ham'
 'spam' 'ham' 'spam' 'spam' 'spam' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham'
 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'spam'
 'spam' 'spam' 'ham' 'spam' 'ham' 'ham' 'spam' 'spam' 'ham' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam'
 'ham' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'ham' 'ham' 'spam' 'ham'
 'spam' 'spam' 'ham' 'spam' 'ham' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'ham' 'spam' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'spam' 'spam'
 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam'
 'spam' 'ham' 'ham' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'ham' 'spam' 'spam' 'spam' 'ham' 'spam' 'ham' 'ham'

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
print("Accuracy: {:.3f}".format(accuracy_score(etiquetas_prueba, etiquetas_prediccion)))

Accuracy: 0.982


### 9. Implementa el ejercicio en Pycharm